<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/WhisperingWithLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Manual Transcription: see [Ren's Wiki](https://libertymutual.atlassian.net/wiki/spaces/~62fefff51e82e839c250dcae/blog/2023/06/22/835472240/PA+university+video+parts+summary) for detail notes on PA University Video Summary

Following is an attempt to automate the whole process **with Whisper and LLM**

**Dependencies**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install -q pytube
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install tiktoken==0.3.3

import os, re
import torch
from pathlib import Path
# from pytube import YouTube

import whisper
from whisper.utils import get_writer


In [5]:
# !whisper "/content/drive/MyDrive/mp4/PAU_Session1_P1_Feb2021.mp4" --model medium.en

In [6]:
file="/content/drive/MyDrive/mp4/PAU_Session1_P1_Feb2021.mp4"
file_path = Path(file)
print(f"Transcribing file: {file_path}\n")
output_directory = file_path.parent

Transcribing file: /content/drive/MyDrive/mp4/PAU_Session1_P1_Feb2021.mp4



In [7]:
# Use CUDA, if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load the desired model
model = whisper.load_model("medium.en").to(DEVICE)


100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 85.4MiB/s]


✍ Transcribing with Whisper

In [18]:
def transcribe_file(model, file, plain=None, srt=None, vtt=None, tsv=None, download=None):
    file_path = Path(file)
    print(f"Transcribing file: {file_path}\n")

    output_directory = file_path.parent

    # Run Whisper
    result = model.transcribe(file, verbose = False, language = "en")
    txt_path = file_path.with_suffix(".txt")
    print(f"\nCreating text file")
    with open(txt_path, "w", encoding="utf-8") as txt:
      txt.write(result["text"])

    # here is more fomalization for any types
    if plain:
        txt_path = file_path.with_suffix(".txt")
        print(f"\nCreating text file")

        with open(txt_path, "w", encoding="utf-8") as txt:
            txt.write(result["text"])
    if srt:
        print(f"\nCreating SRT file")
        srt_writer = get_writer("srt", output_directory)
        srt_writer(result, str(file_path.stem))

    if vtt:
        print(f"\nCreating VTT file")
        vtt_writer = get_writer("vtt", output_directory)
        vtt_writer(result, str(file_path.stem))

    if tsv:
        print(f"\nCreating TSV file")

        tsv_writer = get_writer("tsv", output_directory)
        tsv_writer(result, str(file_path.stem))

    if download:
        from google.colab import files

        colab_files = Path("/content")
        stem = file_path.stem

        for colab_file in colab_files.glob(f"{stem}*"):
            if colab_file.suffix in [".txt", ".srt", ".vtt", ".tsv"]:
                print(f"Downloading {colab_file}")
                files.download(str(colab_file))

    return result

💬 Whisper it This block calls transcribe_file 😉

In [19]:
# if input_format == "youtube":
#     # Download the audio stream of the YouTube video
#     print(f"Downloading audio stream: {audio}")
#     audio = download_youtube_audio(file)

#     # Run Whisper on the audio stream
#     result = transcribe_file(model, audio, plain, srt, vtt, tsv, download)
# elif input_format == "gdrive":
#     # Authorize a connection between Google Drive and Google Colab
#     from google.colab import drive
#     drive.mount('/content/drive')

#     # Run Whisper on the specified file
#     result = transcribe_file(model, file, plain, srt, vtt, tsv, download)
# elif input_format == "local":
    # Run Whisper on the specified file

# from google.colab import drive
# drive.mount('/content/drive/MyDrive/mp4')



Do not run folloing if already audio text is acquired from the given videos

In [ ]:
# result = transcribe_file(model, file)

In [ ]:
!pip install transformers datasets evaluate rouge_score langchain

In [9]:
import os
import json

drive_mount_path = '/content/drive'

# Check if the drive is already mounted
if not os.path.ismount(drive_mount_path):
    from google.colab import drive
    drive.mount(drive_mount_path)
else:
    print("Drive is already mounted.")
with open('/content/drive/MyDrive/secrets.json', 'r') as f:
  secrets = json.load(f)
KEY=secrets['hug_read']
os.environ["HUGGINGFACEHUB_API_TOKEN"] = KEY

Drive is already mounted.


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
from langchain.document_loaders import TextLoader

LOCAL_DRIVR = '/content/drive/MyDrive/mp4'
MY_FILE=LOCAL_DRIVR+"/PAU_Session1_P1_Feb2021.txt"

def loadTXTFileFromLocal(fileName=MY_FILE):
  loader = TextLoader(fileName)
  loaded_docs = loader.load()
  return loaded_docs

In [20]:
from transformers import pipeline

# summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
# summarizer(loadTXTFileFromLocal())

